# Enhancer Prediction Pitfalls Demo

This notebook demonstrates several potential pitfalls of machine learning in genomics, utilizing Python and the scikit-learn machine learning package.  Broad discussion of these pitfalls can be found in our Nature Reviews Genetics paper (link).  Some are well known and not specific to genomics, while others have particular relevance in the field and may be under-appreciated.  The latter are primarily characterized by violations of the Independent and Identically-Distributed (IID) assumption made by most statistical models; see [The Elements of Statistical Learning](https://web.stanford.edu/~hastie/ElemStatLearn/) for a detailed introduction.

The task is to build a binary classifier for enhancers in the K562 cell line.  There are many potential sources of enhancer labels and features for such a problem; our formulation is as follows.  Rows of the feature matrix are [FANTOM5 Capped Analysis of Gene Expression (CAGE)](https://fantom.gsc.riken.jp/5/datafiles/reprocessed/hg38_latest/extra/enhancer/) genomic regions.  Columns are binary features encoding if the region overlaps a statistically significant ChIP-seq peak for a histone modification or transcription factor (TF) assayed by ENCODE in K562.  A region is labeled 1 if its CAGE expression was non-zero in K562, and 0 if there was no expression in K562.

One could use this classifier to make predictions in a related cell line where CAGE has not been performed, or to examine what features are most predictive of K562 CAGE-defined enhancers.  Using known biology, we would expect activating histone marks and TFs to have large positive coefficients in the model, and repressive histone marks and TFs to have large negative coefficients.

The notebook will first load features and labels, define several scikit-learn objects needed to demonstrate various pitfalls, and finally use those objects to fit models and evaluate performance.  Multiple performance metrics will be stored after each evaluation and summarized at the end for simple comparison.

#### - Sean Whalen, Gladstone Institutes, Pollard Lab

In [50]:
import pandas as pd

from sklearn.dummy import *
from sklearn.feature_selection import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import *
from statsmodels.stats.proportion import proportions_ztest

For simplicity, the feature matrix was pre-generated and is loaded from a Feather file, which is a fast binary format for column-based data.  The hg38 genomic coordinates for each region are stored in a column named `coord`, which is later set as the index of the data frame so that the columns only contain features.

To better demonstrate some pitfalls, we filter the data to include regions from chromosomes 1 and 2 only.  Any pair of chromosomes could be used.

In [51]:
features = pd.read_feather('cage-k562-features-hg38.feather')
features = features[features['coord'].str.contains(r'chr[12]:')]
features.set_index('coord', inplace = True)
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10959 entries, chr1:905311-906011 to chr2:242086009-242086316
Columns: 435 entries, ChIP-seq, ADNP, K562, ENCFF817ARM (ENCODE) to ChIP-seq, ZZZ3, K562, ENCFF945HJR (ENCODE)
dtypes: uint8(435)
memory usage: 4.6+ MB


Let's look at a small subset of rows and columns to get a better picture.

In [52]:
features.iloc[10:15, 5:7]

,"ChIP-seq, ARID1B, K562, ENCFF249TYS (ENCODE)","ChIP-seq, ARID2, K562, ENCFF344MKI (ENCODE)"
coord,,
chr1:1020838-1021129,1,0
chr1:1021167-1021519,1,0
chr1:1031105-1031473,0,0
chr1:1069912-1070292,0,0
chr1:1073342-1073506,0,0


Labels were also pre-generated by thresholding the K562 column of CAGE expression values, resulting in a relatively small percent of CAGE peaks with non-zero expression.  This will later demonstrate the impact of class imbalance.

Labels also use genomic coordinates for their index, and re-indexed so their row order matches that of the features.

In [53]:
labels = (
    pd.read_csv('cage-k562-labels-hg38.csv', index_col = 0, squeeze = True)
    .reindex(features.index)
)

In [61]:
labels.sample(10, random_state = 4)

coord
chr1:242523218-242523397    0
chr1:143699615-143699920    1
chr1:112358903-112359157    0
chr1:29209107-29209221      0
chr1:234504245-234504517    0
chr2:234795448-234795858    0
chr2:171141169-171141440    1
chr2:181787807-181788052    0
chr1:224736861-224737237    0
chr2:37768584-37769049      0
Name: k562, dtype: int64

In [62]:
labels.value_counts()

0    10542
1      417
Name: k562, dtype: int64

Next, we extract just the chromosome of each region to later demonstrate the effect of cross-chromosome validation versus k-fold cross-validation with random shuffling.  This uses a regular expression to capture the text before a colon, for example the `chr1` in `chr1:110404708-110405059`.

In [63]:
chroms = (
    features
    .index
    .to_series()
    .str.extract('^([^:]+)')
)

We will define two types of cross-validation, starting with stratified k-fold.  When later passed to a cross-validation function, regions will be randomly shuffled without respect to the chromosome they are located on, and then split into two training/test splits while attempting to presereve the proportion of classes.

In [64]:
shuffled_cv = StratifiedKFold(
    n_splits = 2,
    shuffle = True,
    random_state = 0
)

We next define cross-chromosome validation, which splits the data into two training/test sets where all regions on the same chromosome will be in one set or the other.  This prevents non-independent regions on the same chromosome from inflating performance by being present in both the training and test sets, which violates the Independent and Identically Distributed (IID) assumption of most statistical models.

In [65]:
cc_cv = GroupKFold(n_splits = 2)

We then define multiple performance metrics for later use.

In [66]:
scorers = ['average_precision', 'roc_auc', 'accuracy']

We next create several "pipelines" which define a series of steps used to process data and fit a model.  scikit-learn commonly uses `estimator` as the variable name for pipelines, so we follow this convention.

`VarianceThreshold` is always our first step and removes zero-variance features that generate warnings or cause problems for some models.

Our baseline estimator uses a `DummyClassifier` to always predict the most common class (in our case, 0), and is useful to determine if our statistical model is actually learning anything from the data.

In [67]:
baseline_estimator = make_pipeline(
    VarianceThreshold(),
    DummyClassifier(strategy = 'most_frequent')
)

Our linear model estimator removes zero-variance features, then standardizes each feature by subtracting its mean and dividing by its standard deviation, and finally fits a linear model that uses a L2 ("ridge") penalty to shrink coefficients.

In [68]:
linear_estimator = make_pipeline(
    VarianceThreshold(),
    StandardScaler(),
    RidgeClassifier()
)

We also create a modified linear estimator that pre-selects a small percentage of features based on their correlation with the labels.  Non-selected features are removed before model fitting.  Selection is performed "inside" the cross-validation loop.

In [69]:
selection_percentile = 5
selection_lm_estimator = make_pipeline(
    VarianceThreshold(),
    SelectPercentile(percentile = selection_percentile),
    StandardScaler(),
    RidgeClassifier()
)

Similar to above, but without the model-fitting step.  This will demonstrate a pitfall where features are first selected prior to (or "outside of") cross-validation.

In [70]:
selection_estimator = make_pipeline(
    VarianceThreshold(),
    SelectPercentile(percentile = selection_percentile)
)

We'll store each model's performance in a list for comparison at the end.

In [71]:
all_scores = []

Evaluate the baseline model using shuffled K-fold CV.

In [72]:
scores = cross_validate(
    baseline_estimator,
    features,
    labels,
    groups = chroms,
    cv = shuffled_cv,
    scoring = scorers
)

all_scores.append([
    'none',
    'none',
    'shuffled',
    'baseline',
    scores['test_average_precision'].mean(),
    scores['test_roc_auc'].mean(),
    scores['test_accuracy'].mean()
])

Evaluate the baseline model using cross-chromosome validation.

In [73]:
scores = cross_validate(
    baseline_estimator,
    features,
    labels,
    groups = chroms,
    cv = cc_cv,
    scoring = scorers
)

all_scores.append([
    'none',
    'none',
    'cross-chromosome',
    'baseline',
    scores['test_average_precision'].mean(),
    scores['test_roc_auc'].mean(),
    scores['test_accuracy'].mean()
])

Evaluate the linear model using shuffled K-fold CV.

In [74]:
scores = cross_validate(
    linear_estimator,
    features,
    labels,
    groups = chroms,
    cv = shuffled_cv,
    scoring = scorers
)

all_scores.append([
    'none',
    'none',
    'shuffled',
    'linear',
    scores['test_average_precision'].mean(),
    scores['test_roc_auc'].mean(),
    scores['test_accuracy'].mean()
])

Evaluate the linear model using cross-chromosome validation.

In [75]:
scores = cross_validate(
    linear_estimator,
    features,
    labels,
    groups = chroms,
    cv = cc_cv,
    scoring = scorers
)

all_scores.append([
    'none',
    'none',
    'cross-chromosome',
    'linear',
    scores['test_average_precision'].mean(),
    scores['test_roc_auc'].mean(),
    scores['test_accuracy'].mean()
])

Evaluate the linear model using feature selection inside the cross-validation loop, using cross-chromosome validation.

In [76]:
scores = cross_validate(
    selection_lm_estimator,
    features,
    labels,
    groups = chroms,
    cv = cc_cv,
    scoring = scorers
)

all_scores.append([
    'none',
    'feature selection',
    'cross-chromosome',
    'linear',
    scores['test_average_precision'].mean(),
    scores['test_roc_auc'].mean(),
    scores['test_accuracy'].mean()
])

Pre-select features before performing CV, then evaluate using a linear model with the selected features using cross-chromosome validation.

In [77]:
selected_features = selection_estimator.fit_transform(features, labels)

scores = cross_validate(
    linear_estimator,
    selected_features,
    labels,
    groups = chroms,
    cv = cc_cv,
    scoring = scorers
)

all_scores.append([
    'feature selection',
    'none',
    'cross-chromosome',
    'linear',
    scores['test_average_precision'].mean(),
    scores['test_roc_auc'].mean(),
    scores['test_accuracy'].mean()
])

"Balance" classes by selecting an equal number of positive and negative samples outside of CV.  Requires re-ordering features to match balanced labels, and re-extracting chromosome information.

Next, evaluate a linear model using balanced data and cross-chromosome validation.

In [78]:
minority_class_count = labels.value_counts().min()
balanced_labels = (
    labels
    .groupby(labels, group_keys = False)
    .apply(
        lambda x: x.sample(
            n = minority_class_count,
            random_state = 0
        )
    )
)

balanced_features = features.reindex(balanced_labels.index)

balanced_chroms = (
    balanced_features
    .index
    .to_series()
    .str.extract('^([^:]+)')
)

scores = cross_validate(
    linear_estimator,
    balanced_features,
    balanced_labels,
    groups = balanced_chroms,
    cv = cc_cv,
    scoring = scorers
)

all_scores.append([
    'balance classes',
    'none',
    'cross-chromosome',
    'linear',
    scores['test_average_precision'].mean(),
    scores['test_roc_auc'].mean(),
    scores['test_accuracy'].mean()
])

Combine two pitfalls: select features using balanced data, both outside of CV.

Evaluate a linear model using pre-balanced and pre-feature-selected data using cross-chromosome validation.

In [79]:
selected_balanced_features = selection_estimator.fit_transform(balanced_features, balanced_labels)

scores = cross_validate(
    linear_estimator,
    selected_balanced_features,
    balanced_labels,
    groups = balanced_chroms,
    cv = cc_cv,
    scoring = scorers
)

all_scores.append([
    'balance classes, feature selection',
    'none',
    'cross-chromosome',
    'linear',
    scores['test_average_precision'].mean(),
    scores['test_roc_auc'].mean(),
    scores['test_accuracy'].mean()
])

Summarize our performance metrics across all evaluation settings.

In [80]:
pd.DataFrame(
    all_scores,
    columns = [
        'outside cv steps',
        'inside cv steps',
        'cv type',
        'model type',
        'mean auPR',
        'mean auROC',
        'mean accuracy'
    ]
)

,outside cv steps,inside cv steps,cv type,model type,mean auPR,mean auROC,mean accuracy
0,none,none,shuffled,baseline,0.038051,0.500000,0.961949
1,none,none,cross-chromosome,baseline,0.037744,0.500000,0.962256
2,none,none,shuffled,linear,0.348435,0.756251,0.961858
3,none,none,cross-chromosome,linear,0.287322,0.739716,0.961599
4,none,feature selection,cross-chromosome,linear,0.431820,0.835089,0.968065
5,feature selection,none,cross-chromosome,linear,0.454918,0.841561,0.968059
6,balance classes,none,cross-chromosome,linear,0.684962,0.628195,0.680029
7,"balance classes, feature selection",none,cross-chromosome,linear,0.817434,0.822572,0.789038


This summary showcases many pitfalls of machine learning, both in general and with respect to applications in genomics.

First, note that accuracy is typically very high when area under the precision-recall (auPR) or the receiver operating characteristic (auROC) curves are low.  This is characteristic of problems with major class imbalance, as accuracy simply measures the percent of predictions that are correct without respect to their proportion in the dataset.  Thus, a "model" that always predicts 0 when 96% of the labels are 0 will have an accuracy of 96%.  However, it will never correctly predict the minority class, and this is often the class of interest in genomics.

Also note that auROC can be high when auPR is low.  The ROC curve is formed by values of the true positive rate (tpr) and false positive rate (fpr) at different classifier thresholds:

$$tpr = \frac{tp}{tp + fn}$$
$$fpr = \frac{fp}{fp + tn}$$

Where tp = # true positives, fp = # false positives, tn = # true negatives, and fn = # false negatives.  Precision and recall are defined as:

$$\mathrm{precision} = \frac{tp}{tp + fp}$$
$$\mathrm{recall} = tpr = \frac{tp}{tp + fn}$$

For highly imbalanced problems in genomics, the majority negative class is often much easier to predict than the minority positive class.  In such cases, the number of true negatives will be high and drive the false positive rate very low, impacting auROC.  True negatives are not measured by precision or recall, so auPR is not inflated by easy-to-predict majority classes.  This makes auPR a better choice than auROC for some (but not all) problems in genomics.

Second, feature selection performed outside CV leads to higher performance compared to selection performed inside CV.  The improvement here is not drastic, but often is; this estimator selects a percentage of features based on their individual correlations with the class label, rather than considering features jointly.  Selecting features using a multivariate linear model with an L1 penalty outside CV leads to an even larger gap.  Another procedure performed outside CV balances the number of positive and negative classes by downsampling the majority class.  This drastically inflates performance, especially when combined with feature selection; the resulting auPR of 0.81 is nearly twice that of the same model evaluated correctly.

Third, note that shuffled k-fold CV reports higher auPR than cross-chromosome validation.  Regions on the same chromosome may not be independent and thus have highly correlated features (violating independence in the IID assumption), making it easier for a model that trains one sample to accurately predict the label of another correlated sample.  Shuffling samples and creating training/test splits that ignore these potential correlations can inflate performance.  Using cross-cromosome validation, where all samples on the same chromosome are placed in either the training or test set, prevents this type of inflation.

Samples might also violate the identically-distributed portion of the IID assumption, having features that are distributed differently across chromosomes due to the underlying biology.  In this example, samples are not highly correlated; we will show the performance difference is due such such differences in feature distribution.  We can test this by building an adversarial classifier that tries to predict which chromosome a sample originates from, rather than trying to predict the enhancer label.

In [81]:
chromosome_labels = (
    features
    .index
    .to_series()
    .str.startswith('chr1:')
    .astype(int)
)

scores = cross_val_score(
    linear_estimator,
    features,
    chromosome_labels,
    cv = 5,
    scoring = 'average_precision'
)
print(f'adversarial performance: {scores.mean():.2f}')

adversarial performance: 0.58


This is a modest improvement over random guessing, suggesting that the performance drop observed using cross-chromosome evaluation is at least partly due to differences in the distribution of features.

Since our features are binary, feature density plots won't be very interesting.  Instead, let's take a statistical approach and perform a difference-of-proportions test for the values of the most predictive features, split by chromosome.

In [82]:
linear_estimator.fit(features, labels)

coefs = pd.Series(
    linear_estimator.steps[-1][1].coef_[0],
    index = features.columns[linear_estimator.steps[0][1].get_support()]
)
coefs = (
    coefs
    .abs()
    .sort_values(ascending = False)
    .index
    .tolist()
)

features['chrom'] = chromosome_labels

for i in range(5):
    counts = (
        features
        .groupby('chrom')
        [coefs[i]]
        .agg([sum, len])
    )

    zstat, pvalue = proportions_ztest(counts['sum'], counts['len'])
    print(f'{zstat:.1f} {pvalue:.1e} {coefs[i]}')

-6.2 6.0e-10 ChIP-seq, ARNT, K562, ENCFF447FIO (ENCODE)
-5.3 1.4e-07 ChIP-seq, POLR2A, K562, ENCFF285MBX (ENCODE)
-5.0 7.1e-07 ChIP-seq, POLR2A, K562, ENCFF099NYA (ENCODE)
-4.4 1.0e-05 ChIP-seq, TAF1, K562, ENCFF453TIB (ENCODE)
-5.1 4.2e-07 ChIP-seq, POLR2A, K562, ENCFF730DLS (ENCODE)


The p-value for the difference-of-proportions test is significant for the top 5 features.  Repeating this test for the least predictive features shows no significant p-values.  In summary, features the model finds most predictive are also distributed somewhat differently across chromosomes, causing a moderate drop in performance when the model is forced to train on samples from only one chromosome and predict on another.